In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet,ElasticNetCV
from sklearn.metrics import mean_squared_error
pd.options.display.max_columns = 30


In [18]:
df1 = pd.read_csv('interpolate_all_uv_data.csv', index_col = 0)
df2 = pd.read_csv('remove_uv_data.csv', index_col = 0)

In [19]:
df1 = df1.drop(['날짜','시간_분','관측소지점','경도','위도','계절'], axis = 1)
df2 = df2.drop(['시간_분','관측소지점','경도','위도','계절'], axis = 1)

In [46]:
df1 = pd.get_dummies(df1, columns = ['지면타입'])
df2 = pd.get_dummies(df2, columns = ['지면타입'])

# 자외선 결측치  채운 데이터

In [47]:
df1.describe()

,자외선지수,파랑_가시밴드,초록_가시밴드,빨강_가시밴드,식생_가시밴드,권운_밴드,눈/얼음_가시밴드,야간안개/하층운_밴드,상층_수증기_밴드,중층_수증기_밴드,...,이산화탄소_밴드,태양천정각,위성천정각,대기외일사량,관측고도,월,지면타입_0,지면타입_2,지면타입_3,지면타입_4
count,1.578960e+06,1.578960e+06,1.578960e+06,1.578960e+06,1.578960e+06,1.578960e+06,1.578960e+06,1.578960e+06,1.578960e+06,1.578960e+06,...,1.578960e+06,1.578960e+06,1.578960e+06,1.578960e+06,1.578960e+06,1.578960e+06,1578960.0,1.578960e+06,1.578960e+06,1.578960e+06
mean,1.083512e+00,9.930942e-02,9.565025e-02,8.877412e-02,1.056360e-01,1.198601e-02,6.812384e-02,2.793715e+02,2.367864e+02,2.448662e+02,...,2.580409e+02,8.898176e+01,4.211422e+01,-3.938039e-02,6.863933e+01,6.519836e+00,0.2,6.666667e-02,6.666667e-01,6.666667e-02
std,1.962107e+00,1.523718e-01,1.517827e-01,1.521736e-01,1.693914e-01,3.763262e-02,1.024260e-01,1.676730e+01,6.306382e+00,7.911280e+00,...,1.267756e+01,3.806119e+01,1.339209e+00,5.437120e+00,4.635743e+01,3.449552e+00,0.4,2.494439e-01,4.714047e-01,2.494439e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.527700e+02,1.964636e+02,1.952607e+02,...,1.962145e+02,9.868750e+00,3.896359e+01,-9.630860e+00,2.280000e+00,1.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.685674e+02,2.329808e+02,2.408254e+02,...,2.514128e+02,5.856060e+01,4.104849e+01,-4.806660e+00,4.700000e+01,4.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,9.630000e-03,7.980000e-03,4.770000e-03,5.540000e-03,0.000000e+00,1.930000e-03,2.808032e+02,2.371707e+02,2.460045e+02,...,2.608939e+02,8.885167e+01,4.202608e+01,1.785500e-02,6.894000e+01,7.000000e+00,0.0,0.000000e+00,1.000000e+00,0.000000e+00
75%,1.300000e+00,1.444600e-01,1.328700e-01,1.111600e-01,1.539700e-01,2.720000e-03,1.148100e-01,2.915966e+02,2.411857e+02,2.502092e+02,...,2.675652e+02,1.198613e+02,4.367046e+01,4.845467e+00,7.238000e+01,1.000000e+01,0.0,0.000000e+00,1.000000e+00,0.000000e+00
max,2.360000e+01,1.052540e+00,1.068690e+00,1.122040e+00,1.161110e+00,6.089700e-01,7.107700e-01,3.212326e+02,2.626168e+02,2.721262e+02,...,2.815333e+02,1.700420e+02,4.395463e+01,9.025490e+00,2.228000e+02,1.200000e+01,1.0,1.000000e+00,1.000000e+00,1.000000e+00


In [33]:
X = df1.drop(['자외선지수'], axis = 1)
y = df1['자외선지수']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=50)

In [28]:
alphas = [0.01,0.1, 0.3, 0.5, 0.7]
for a in alphas:
    model = ElasticNet(alpha=a).fit(X,y)   
    score = model.score(X, y)
    pred_y = model.predict(X)
    mse = mean_squared_error(y, pred_y)   
    print("Alpha:{0:.4f}, R2:{1:.2f}, MSE:{2:.2f}"
          .format(a, score, mse))

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.881e+05, tolerance: 6.079e+02
  model = cd_fast.enet_coordinate_descent(


Alpha:0.0100, R2:0.63, MSE:1.43


C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.819e+03, tolerance: 6.079e+02
  model = cd_fast.enet_coordinate_descent(


Alpha:0.1000, R2:0.61, MSE:1.49
Alpha:0.3000, R2:0.59, MSE:1.56
Alpha:0.5000, R2:0.58, MSE:1.61
Alpha:0.7000, R2:0.57, MSE:1.65


### a = 0.01일 시 제일 좋은 모형

In [42]:
alphas = np.arange(0.01, 0.9, step=0.01)


elastic_cv=ElasticNetCV(alphas=alphas)
model = elastic_cv.fit(X_train,y_train)
print(model.alpha_)

y_pred = model.predict(X_test)
score = model.score(X_test, y_test)
mse = mean_squared_error(y_test, y_pred)
print("R2:{0:.3f}, MSE:{1:.2f}"
      .format(score, mse))


C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 801.8340489256661, tolerance: 291.70118877281453
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4666.131952117663, tolerance: 291.70118877281453
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12170.487208087347, tolerance: 291.70118877281453
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.098e+05, tolerance: 3.644e+02
  model = cd_fast.enet_coordinate_descent(


0.01
R2:0.628, MSE:1.44


# 자외선 결측치 제거 데이터

In [48]:
df2.describe()

,자외선지수,파랑_가시밴드,초록_가시밴드,빨강_가시밴드,식생_가시밴드,권운_밴드,눈/얼음_가시밴드,야간안개/하층운_밴드,상층_수증기_밴드,중층_수증기_밴드,...,이산화탄소_밴드,태양천정각,위성천정각,대기외일사량,관측고도,월,지면타입_0,지면타입_2,지면타입_3,지면타입_4
count,1.525753e+06,1.525753e+06,1.525753e+06,1.525753e+06,1.525753e+06,1.525753e+06,1.525753e+06,1.525753e+06,1.525753e+06,1.525753e+06,...,1.525753e+06,1.525753e+06,1.525753e+06,1.525753e+06,1.525753e+06,1.525753e+06,1.525753e+06,1.525753e+06,1.525753e+06,1.525753e+06
mean,1.096784e+00,1.003195e-01,9.664504e-02,8.973907e-02,1.068459e-01,1.211955e-02,6.880207e-02,2.795062e+02,2.367935e+02,2.448724e+02,...,2.580833e+02,8.861417e+01,4.210759e+01,1.223589e-02,6.852743e+01,6.566434e+00,2.010142e-01,6.808835e-02,6.631735e-01,6.772394e-02
std,1.976032e+00,1.530552e-01,1.524948e-01,1.529397e-01,1.702420e-01,3.778310e-02,1.028394e-01,1.680709e+01,6.321464e+00,7.935523e+00,...,1.272678e+01,3.805311e+01,1.342000e+00,5.433881e+00,4.675649e+01,3.421990e+00,4.007588e-01,2.518975e-01,4.726252e-01,2.512717e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.527700e+02,1.964636e+02,1.952607e+02,...,1.962145e+02,9.868750e+00,3.896359e+01,-9.630860e+00,2.280000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.687980e+02,2.329808e+02,2.407902e+02,...,2.513926e+02,5.826880e+01,4.104849e+01,-4.764510e+00,4.700000e+01,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,1.246000e-02,1.026000e-02,6.560000e-03,7.480000e-03,0.000000e+00,3.220000e-03,2.810092e+02,2.371707e+02,2.460351e+02,...,2.610234e+02,8.832422e+01,4.202608e+01,1.346400e-01,6.894000e+01,7.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
75%,1.400000e+00,1.455900e-01,1.340100e-01,1.126500e-01,1.560600e-01,2.720000e-03,1.161000e-01,2.917323e+02,2.411857e+02,2.502642e+02,...,2.676526e+02,1.195898e+02,4.367046e+01,4.883570e+00,7.238000e+01,1.000000e+01,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
max,2.360000e+01,1.052540e+00,1.068690e+00,1.122040e+00,1.161110e+00,6.089700e-01,7.107700e-01,3.212326e+02,2.626168e+02,2.721262e+02,...,2.815333e+02,1.700420e+02,4.395463e+01,9.025490e+00,2.228000e+02,1.200000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [43]:
X = df2.drop(['자외선지수'], axis = 1)
y = df2['자외선지수']

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=50)

In [32]:
alphas = [0.01,0.1, 0.3, 0.5, 0.7]
for a in alphas:
    model = ElasticNet(alpha=a).fit(X,y)   
    score = model.score(X, y)
    pred_y = model.predict(X)
    mse = mean_squared_error(y, pred_y)   
    print("Alpha:{0:.4f}, R2:{1:.2f}, MSE:{2:.2f}"
          .format(a, score, mse))

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.766e+05, tolerance: 5.958e+02
  model = cd_fast.enet_coordinate_descent(


Alpha:0.0100, R2:0.63, MSE:1.44


C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.322e+03, tolerance: 5.958e+02
  model = cd_fast.enet_coordinate_descent(


Alpha:0.1000, R2:0.61, MSE:1.51
Alpha:0.3000, R2:0.60, MSE:1.57
Alpha:0.5000, R2:0.58, MSE:1.62
Alpha:0.7000, R2:0.57, MSE:1.67


### a = 0.01일 시 제일 좋은 모형

In [45]:
alphas = np.arange(0.01, 0.9, step=0.01)

elastic_cv=ElasticNetCV(alphas=alphas)
model = elastic_cv.fit(X_train,y_train)
print(model.alpha_)

y_pred = model.predict(X_test)
score = model.score(X_test, y_test)
mse = mean_squared_error(y_test, y_pred)
print("R2:{0:.3f}, MSE:{1:.2f}"
      .format(score, mse))


C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1112.2716292550322, tolerance: 286.24859228220566
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4929.1943778777495, tolerance: 286.24859228220566
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12119.928222073824, tolerance: 286.24859228220566
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not conver

0.01
R2:0.630, MSE:1.44


C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.044e+05, tolerance: 3.579e+02
  model = cd_fast.enet_coordinate_descent(
